In [1]:
from caveclient import CAVEclient

# List available annotation tables

In [2]:
url = "https://global.brain-wire-test.org/"

# For H01
# datastack = "h01_c3_flat"
# voxel_resolution = [8, 8, 33]

# For Fish1
datastack = "fish1_full"
voxel_resolution = [16, 16, 30]

In [5]:
client = CAVEclient(datastack_name=datastack, server_address=url)
client.info.get_datastack_info()

{'aligned_volume': {'id': 4,
  'name': 'fish1',
  'display_name': None,
  'description': 'aligned volume image stack of Fish1',
  'image_source': 'precomputed://gs://fish1-public/clahe_231218'},
 'segmentation_source': 'graphene://https://pcgv3local.brain-wire-test.org/segmentation/table/fish1_v250117',
 'skeleton_source': 'precomputed://https://pcgv3local.brain-wire-test.org/skeletoncache/api/v1/fish1_full/precomputed/skeleton/',
 'analysis_database': None,
 'viewer_site': 'https://ngl.brain-wire.org/',
 'synapse_table': 'synapses_axax',
 'soma_table': None,
 'local_server': 'https://pcgv3local.brain-wire-test.org',
 'description': 'datastack with latest agglomeration (v250117)',
 'viewer_resolution_x': 16.0,
 'viewer_resolution_y': 16.0,
 'viewer_resolution_z': 30.0,
 'proofreading_status_table': None,
 'cell_identification_table': None,
 'proofreading_review_table': None}

In [4]:
# get existing tables
all_tables = client.annotation.get_tables()
all_tables

['synapses_axax', 'synapses_axde']

## Create a nucleus table
To create a table a description and a table schema is required. A list of available schemas can be found [here](https://global.brain-wire-test.org/schema/views/). 


In [16]:
# create a new table
table_name = "nucleus_table"
description = """
This is a table that contains nucleus id, volume, position and bounding box's position.

About Columns:
- id: ID in the VAST segmentation file (for neurons/glia this should be consistent with the 'dbid' value in other tables)
- volume: volume estimate of cell body or nucleus in cubic micrometers, at _very low resolution_ (mip6 every 128th section for neurons/glia, mip7 every 16th section for bv nuclei, manually painted)
- pt_position: X,Y,Z anchor point within cell body or nucleus, in pixels at 8x8x33 nm
- bb_start_position: Xmin,Ymin,Zmin of bounding box of cell body or nucleus, in pixels at 8x8x33 nm
- bb_end_position: Xmax,Ymax,Zmax of bounding box of cell body or nucleus, in pixels at 8x8x33 nm
The data in this table was compiled on May 16, 2023, by Daniel Berger.

This table was create by Jinhan Choi.
"""
schema_name = "nucleus_detection"

In [ ]:
client.annotation.create_table(table_name=table_name, 
                               schema_name=schema_name, 
                               description=description, 
                               voxel_resolution=voxel_resolution)

<Response [200]>

## Create annotation table

In [18]:
# create a new table
table_name = "dbcells_dump"
description = """
This is a table that contains cell's position, types, and sub-types.

About Columns:
- pt_position: X,Y,Z anchor point within cell body or nucleus, in pixels at 8x8x33 nm
- classification_system: comma-seperated string joined from 'dbsubcelltype', 'dbsubcelltype2', and 'dbsubcelltype3' 
- cell_type:  cell type (manually classified):
% dbcelltypes are:
%   0:  [U] unknown,
%   1:  [P  pyramidal,
%   2:  [I  interneuron,
%   3:  [N  unclassified neuron,
%   4:  [A] astrocyte,
%   5:  [O] oligodendrocyte,
%   6:  [G] mg/opc,
%   7:  [C] 'c-shaped' cell (also mg/opc),
%   8:  [B] blood vessel cell (does not occur in this list)
%   9:  [S  Spiny stellate
%   10: [E  Excitatory/spiny with atypical tree
% Evelinas blood vessel nucleus types are:
%   11: 'Pericytes'
%   12: 'vSMC'
%   13: 'fibroblast'
%   14: 'perivascular lymphocytes'
%   15: 'perivascular macrophages'
%   16: 'undecided' (unspecified blood vessel cell)
%   17: 'Endothelial cells'
%   18: 'circulating imm'

The data in this table was compiled on May 16, 2023. 
Source: console.cloud.google.com/bigquery?ws=!1m5!1m4!4m3!1slcht-goog-connectomics!2sgoog14r0seg1!3sdbcells

This table was create by Jinhan Choi.
"""
schema_name = "cell_type_local"

In [19]:
client.annotation.create_table(table_name=table_name, 
                               schema_name=schema_name, 
                               description=description, 
                               voxel_resolution=[8, 8, 33])

<Response [200]>

In [8]:
all_tables = client.annotation.get_tables()
all_tables

['cells', 'nucleus', 'synapses']